In [4]:
import pandas as pd 


df = pd.read_csv('sintetic\output\latex\data.csv')

df.head()

,formula,path,selection_type
0,b,output/latex\train\image_6.png,train
1,v,output/latex\train\image_1.png,train
2,\exists! z: Q(z),output/latex\train\image_8.png,train
3,"\int_{-\infty}^{b} ((a)) \, dy",output/latex\train\image_13.png,train
4,\{ a \in \varnothing \mid {e}^{0} \geq {v}^{a}...,output/latex\train\image_5.png,train


In [9]:
import pandas as pd
import requests
import base64
from PIL import Image
import json
import time

# Конфигурация
LM_STUDIO_URL = "http://localhost:1111/v1/chat/completions"
CSV_PATH = "sintetic\output\latex\data.csv"
RESULTS_PATH = "results.csv"

def encode_image(image_path):
    """Кодирование изображения в base64"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def query_lm_studio(image_path, prompt_template):
    """Отправка запроса к LM Studio API"""
    base64_image = encode_image(image_path)
    
    headers = {"Content-Type": "application/json"}
    
    # Формируем сообщение с изображением и текстом
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                },
                {
                    "type": "text",
                    "text": prompt_template
                }
            ]
        }
    ]
    
    payload = {
        "messages": messages,
        "temperature": 0.1,
        "max_tokens": 512,
        "stream": False
    }
    
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"Ошибка при запросе: {e}")
        return None

def evaluate_predictions(true_latex, predicted_latex):
    """Простая оценка точности предсказания"""
    # Удаляем пробелы и переносы для сравнения
    true_clean = true_latex.replace('\n', '').replace(' ', '')
    pred_clean = predicted_latex.replace('\n', '').replace(' ', '')
    
    return true_clean == pred_clean

def main():
    # Загружаем данные
    df = pd.read_csv(CSV_PATH)
    
    results = []
    
    # Промпт для модели (можете настроить под свои нужды)
    prompt_template = """Extract the LaTeX code from this mathematical expression image. 
    Return ONLY the LaTeX code without any explanations, markdown, or additional text.
    LaTeX code:"""
    
    print(f"Начинаю обработку {len(df)} изображений...")
    
    for idx, row in df.iterrows():
        image_path = 'sintetic\output\latex\\' + '\\'.join(row['path'].split('\\')[-2::]) 
        true_latex = row['formula']  # предполагаемое имя колонки
        
        print(f"Обработка {idx+1}/{len(df)}: {image_path}")
        
        # Получаем предсказание от модели
        predicted_latex = query_lm_studio(image_path, prompt_template)
        
        if predicted_latex:
            # Оцениваем результат
            is_correct = evaluate_predictions(true_latex, predicted_latex)
            
            results.append({
                'image_path': image_path,
                'true_latex': true_latex,
                'predicted_latex': predicted_latex,
                'is_correct': is_correct
            })
            
            print(f"  True: {true_latex[:50]}...")
            print(f"  Pred: {predicted_latex[:50]}...")
            print(f"  Correct: {is_correct}")
        else:
            results.append({
                'image_path': image_path,
                'true_latex': true_latex,
                'predicted_latex': 'ERROR',
                'is_correct': False
            })
        
        # Небольшая задержка, чтобы не перегружать сервер
        time.sleep(0.5)
    
    # Сохраняем результаты
    results_df = pd.DataFrame(results)
    results_df.to_csv(RESULTS_PATH, index=False)
    
    # Выводим статистику
    if len(results) > 0:
        accuracy = results_df['is_correct'].mean() * 100
        print(f"\nТочность: {accuracy:.2f}%")
        print(f"Обработано: {len(results_df)} изображений")
        print(f"Результаты сохранены в {RESULTS_PATH}")

if __name__ == "__main__":
    main()

Начинаю обработку 6547 изображений...
Обработка 1/6547: sintetic\output\latex\train\image_6.png
  True: b...
  Pred: \[
b
\]...
  Correct: False
Обработка 2/6547: sintetic\output\latex\train\image_1.png
  True: v...
  Pred: \[
v
\]...
  Correct: False
Обработка 3/6547: sintetic\output\latex\train\image_8.png
  True: \exists! z: Q(z)...
  Pred: \exists!z : Q(z)...
  Correct: True
Обработка 4/6547: sintetic\output\latex\train\image_13.png
  True: \int_{-\infty}^{b} ((a)) \, dy...
  Pred: \[
\int_{-\infty}^{b} ((a)) \, dy
\]...
  Correct: False
Обработка 5/6547: sintetic\output\latex\train\image_5.png
  True: \{ a \in \varnothing \mid {e}^{0} \geq {v}^{a} \}^...
  Pred: \[
\{a \in \varnothing | e^{0} \geq v^{a}\}^{\math...
  Correct: False
Обработка 6/6547: sintetic\output\latex\train\image_2.png
  True: z  \notin  \mathbb{P}  \setminus  \mathbb{C} \time...
  Pred: $z \notin P \setminus C \times A \setminus Z$...
  Correct: False
Обработка 7/6547: sintetic\output\latex\train\image_10.png


KeyboardInterrupt: 